In [49]:
import xarray
import xarray as xr
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import cm
import matplotlib.lines as mlines
import cartopy.crs as ccrs
import numpy as np
from plot_common import annotate_skill, autoextend_colorbar, corners, get_map_norm
import xesmf
import subprocess
from typing import Any, Callable, Union
import cartopy.feature as cfeature
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPoint
import regionmask
import cftime
import xskillscore as xs
from sklearn.utils import resample
from multiprocessing import Pool
import useful_functions
import importlib
importlib.reload(useful_functions)

from useful_functions import get_nwa12_outfile, weighted_annual_mean, \
                             area_masked_weighted_average, plot_epu, load_ecodata, \
                             plot_coldpool, compute_gs_points, compute_gs_expts, plot_gs, \
                             plot_epu_dclim, plot_epu_chlos_dclim, plot_epu_mclim, compute_epu_daily_clim

import warnings
warnings.filterwarnings('ignore', 'Converting a CFTimeIndex')
warnings.filterwarnings('ignore')

%matplotlib inline

In [50]:
VARNAME = 'ssh'

if VARNAME=='MLD_003' or VARNAME=='chlos':
    freq='monthly'
else:
    freq='annual'  

hind_start_nwa12 = 1965
hind_end_nwa12 = 2024

hind_start_spear = 1965
hind_end_spear = 2024

clim_start = 1965
clim_end = 2019

hist_strt = 1965
hist_end = 2031
hist_end_long = 2098

epu_shp_file = '/home/acr/git/nwa-shared/NWA12/data/geography/EPU_NOESTUARIES/EPU_NOESTUARIES.shp'
lmes_shp_file = '/home/acr/git/nwa-shared/NWA12/data/geography/lmes/lme66.shp'  

In [51]:
nwa12_hind_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_hindcast_{freq}_{VARNAME}.nc'
ds_nwa12_hind=xr.open_dataset(nwa12_hind_file_path)

if VARNAME=='tos' or VARNAME=='tob':
    nwa12_hist_file_path = f'/work/vnk/outdata_for_analysis/nwa12_historical_{freq}_{VARNAME}.nc'
    spear_hist_file_path = f'/work/vnk/outdata_for_analysis/spear_historical_{freq}_{VARNAME}.nc'
    spear_hind_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_hindcast_{freq}_{VARNAME}.nc'
    ds_nwa12_hist=xr.open_dataset(nwa12_hist_file_path)
    ds_spear_hist=xr.open_dataset(spear_hist_file_path)
    ds_spear_hind=xr.open_dataset(spear_hind_file_path)

if VARNAME=='MLD_003' and freq =='monthly':
    spear_hind_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_hindcast_{freq}_{VARNAME}.nc'
    ds_spear_hind=xr.open_dataset(spear_hind_file_path)

In [52]:
ds_nwa12_hind

<xarray.Dataset> Size: 8GB
Dimensions:  (time: 60, member: 5, lead: 10, yh: 845, xh: 775)
Coordinates:
  * lead     (lead) int64 80B 1 2 3 4 5 6 7 8 9 10
  * xh       (xh) float64 6kB -98.0 -97.92 -97.84 ... -36.24 -36.16 -36.08
  * yh       (yh) float64 7kB 5.273 5.352 5.432 5.511 ... 51.88 51.9 51.91 51.93
  * member   (member) int64 40B 1 2 3 4 5
  * time     (time) datetime64[ns] 480B 1965-01-01 1966-01-01 ... 2024-01-01
Data variables:
    ssh      (time, member, lead, yh, xh) float32 8GB ...

In [53]:
if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':
    print(ds_spear_hind)

In [54]:
%%time

if (VARNAME=='MLD_003' or VARNAME=='chlos') and freq =='monthly':
    
    if VARNAME=='MLD_003':
        selected_months=[1, 2, 3]
    else:
        selected_months=[3, 4, 5]  
        
    # Compute the month index within each year (1 to 12)
    months_within_year = (ds_nwa12_hind['lead'] - 1) % 12 + 1
    
    # Filter the dataset to include only the selected months
    ds_selected_months = ds_nwa12_hind.where(months_within_year.isin(selected_months), drop=True)
    
    # Recompute the years array for the filtered dataset
    years_filtered = ds_selected_months['lead'] // 12
    
    # Group by year and calculate the mean over the selected months
    annual_mean_selected_months = (
        ds_selected_months.groupby(years_filtered)
        .mean(dim='lead')
        .load()
    )
    
    ds_nwa12_hind = annual_mean_selected_months.assign_coords(lead=annual_mean_selected_months['lead'] + 1)

if VARNAME=='MLD_003':

    selected_months=[1, 2, 3]

    # Compute the month index within each year (1 to 12)
    months_within_year = (ds_spear_hind['lead'] - 1) % 12 + 1
    
    # Filter the dataset to include only the selected months
    ds_selected_months = ds_spear_hind.where(months_within_year.isin(selected_months), drop=True)
    
    # Recompute the years array for the filtered dataset
    years_filtered = ds_selected_months['lead'] // 12
    
    # Group by year and calculate the mean over the selected months
    annual_mean_selected_months = (
        ds_selected_months.groupby(years_filtered)
        .mean(dim='lead')
        .load()
    )
    
    ds_spear_hind = annual_mean_selected_months.assign_coords(lead=annual_mean_selected_months['lead'] + 1)

CPU times: user 14 μs, sys: 0 ns, total: 14 μs
Wall time: 23.6 μs


In [55]:
ds_nwa12_hind

<xarray.Dataset> Size: 8GB
Dimensions:  (time: 60, member: 5, lead: 10, yh: 845, xh: 775)
Coordinates:
  * lead     (lead) int64 80B 1 2 3 4 5 6 7 8 9 10
  * xh       (xh) float64 6kB -98.0 -97.92 -97.84 ... -36.24 -36.16 -36.08
  * yh       (yh) float64 7kB 5.273 5.352 5.432 5.511 ... 51.88 51.9 51.91 51.93
  * member   (member) int64 40B 1 2 3 4 5
  * time     (time) datetime64[ns] 480B 1965-01-01 1966-01-01 ... 2024-01-01
Data variables:
    ssh      (time, member, lead, yh, xh) float32 8GB ...

In [56]:
if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':
    print(ds_spear_hind)

In [57]:
%%time

ds_nwa12_hind_ly = []
ds_nwa12_hind_ly_clim = []

for lead in tqdm(ds_nwa12_hind.lead.values):
    
    start_year = hind_start_nwa12 + lead - 1
    end_year = hind_end_nwa12 + lead - 1
    
    ly_time_range = pd.date_range(f'{start_year}-01-01', f'{end_year}-01-01', freq='YS')
    
    nwa12_hind_ly = ds_nwa12_hind.sel(lead=lead).drop_vars('lead').squeeze()
    nwa12_hind_ly['time'] = ly_time_range

    nwa12_hind_ly_clim = nwa12_hind_ly.sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-01-01')).mean(dim=['time'])
    
    nwa12_hind_ly_anom = (nwa12_hind_ly - nwa12_hind_ly_clim)
    nwa12_hind_ly_anom['time'] = pd.date_range(f'{hind_start_nwa12}-01-01', f'{hind_end_nwa12}-01-01', freq='YS')

    ds_nwa12_hind_ly_clim.append(nwa12_hind_ly_clim)
    ds_nwa12_hind_ly.append(nwa12_hind_ly_anom)    
    
ds_nwa12_hind_ly_clim = xr.concat(ds_nwa12_hind_ly_clim, dim='lead')
ds_nwa12_hind_ly_anom = xr.concat(ds_nwa12_hind_ly, dim='lead')

if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':
    
    ds_spear_hind_ly = []
    
    for lead in tqdm(ds_nwa12_hind.lead.values):
    
        start_year = hind_start_spear + lead - 1
        end_year = hind_end_spear + lead - 1
    
        ly_time_range = pd.date_range(f'{start_year}-01-01', f'{end_year}-01-01', freq='YS')
    
        spear_hind_ly = ds_spear_hind.sel(lead=lead).drop_vars('lead').squeeze()
        spear_hind_ly['time'] = ly_time_range
        spear_hind_ly_anom = (spear_hind_ly - spear_hind_ly.sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-01-01')).mean(dim=['time']))
        spear_hind_ly_anom['time'] = pd.date_range(f'{hind_start_spear}-01-01', f'{hind_end_spear}-01-01', freq='YS')
        ds_spear_hind_ly.append(spear_hind_ly_anom)
        
    ds_spear_hind_ly_anom = xr.concat(ds_spear_hind_ly, dim='lead')

if VARNAME=='tos' or VARNAME=='tob':
    ds_nwa12_hist_ym_anom = (ds_nwa12_hist[VARNAME] - ds_nwa12_hist[VARNAME].sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-01-01')).mean(dim=['time'])).to_dataset()
    ds_nwa12_hist_ym_anom['time'] = pd.date_range(f'{hist_strt}-01-01', f'{hist_end}-01-01', freq='YS')

    ds_spear_hist_ym_anom = (ds_spear_hist[VARNAME] - ds_spear_hist[VARNAME].sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-01-01')).mean(dim=['time'])).to_dataset()
    ds_spear_hist_ym_anom['time'] = pd.date_range(f'{hist_strt}-01-01', f'{hist_end}-01-01', freq='YS')

100%|██████████| 10/10 [01:08<00:00,  6.85s/it]


CPU times: user 19.9 s, sys: 34.1 s, total: 54 s
Wall time: 1min 16s


In [58]:
ds_nwa12_hind_ly_anom

<xarray.Dataset> Size: 8GB
Dimensions:  (lead: 10, time: 60, member: 5, yh: 845, xh: 775)
Coordinates:
  * xh       (xh) float64 6kB -98.0 -97.92 -97.84 ... -36.24 -36.16 -36.08
  * yh       (yh) float64 7kB 5.273 5.352 5.432 5.511 ... 51.88 51.9 51.91 51.93
  * member   (member) int64 40B 1 2 3 4 5
  * time     (time) datetime64[ns] 480B 1965-01-01 1966-01-01 ... 2024-01-01
Dimensions without coordinates: lead
Data variables:
    ssh      (lead, time, member, yh, xh) float32 8GB nan nan nan ... nan nan

In [59]:
ds_nwa12_hind_ly_clim

<xarray.Dataset> Size: 131MB
Dimensions:  (lead: 10, member: 5, yh: 845, xh: 775)
Coordinates:
  * xh       (xh) float64 6kB -98.0 -97.92 -97.84 ... -36.24 -36.16 -36.08
  * yh       (yh) float64 7kB 5.273 5.352 5.432 5.511 ... 51.88 51.9 51.91 51.93
  * member   (member) int64 40B 1 2 3 4 5
Dimensions without coordinates: lead
Data variables:
    ssh      (lead, member, yh, xh) float32 131MB nan nan nan ... -1.081 -1.081

In [60]:
if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':
    ds_spear_hind_ly_anom

In [61]:
%%time

if VARNAME=='tob':
    ds_spear_a1 = xr.open_mfdataset(f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_a1_ocean_{freq}.thetao*.nc' \
                               ,concat_dim='member',combine='nested')
    
    ds_spear_a1 = ds_spear_a1.ffill(dim='z_l') \
                      .isel(z_l=-1) \
                      .drop_vars('z_l') \
                      .rename({'thetao': 'tob'}).load()
    
    ds_spear_a1_ym = ds_spear_a1[VARNAME].sel(time=slice('1965', '2023'))
    ds_spear_a1_anom = (ds_spear_a1_ym - ds_spear_a1_ym.sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-07-01')).mean(dim=['time'])).to_dataset(name=f'{VARNAME}')
    ds_spear_a1_anom["time"] = pd.date_range(f'1965-01-01', f'2023-01-01', freq='YS')
    ds_spear_a1_anom

elif VARNAME=='tos':
    ds_spear_a1 = xr.open_mfdataset(f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_a1_ocean_{freq}.tos*.nc' \
                               ,concat_dim='member',combine='nested')
    
    ds_spear_a1 = ds_spear_a1.rename({'SST': 'tos'}).load()
    ds_spear_a1_ym = ds_spear_a1[VARNAME].sel(time=slice('1965', '2023'))
    ds_spear_a1_anom = (ds_spear_a1_ym - ds_spear_a1_ym.sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-07-01')).mean(dim=['time'])).to_dataset(name=f'{VARNAME}')
    ds_spear_a1_anom["time"] = pd.date_range(f'1965-01-01', f'2023-01-01', freq='YS')

    ds_spear_a1_anom

elif VARNAME=='MLD_003':
    ds_spear_a1 = xr.open_mfdataset(f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_a1_ocean_{freq}.{VARNAME}*.nc' \
                               ,concat_dim='member',combine='nested')
    
    months=[1, 2, 3]

    ds_spear_a1 = ds_spear_a1.sel(time=ds_spear_a1.time.dt.month.isin(months)).groupby('time.year').mean(dim='time').load().rename({'year':'time'})
    
    ds_spear_a1_ym = ds_spear_a1[VARNAME].sel(time=slice('1965', '2023'))
    ds_spear_a1_anom = (ds_spear_a1_ym - ds_spear_a1_ym.sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-07-01')).mean(dim=['time'])).to_dataset(name=f'{VARNAME}')
    ds_spear_a1_anom["time"] = pd.date_range(f'1965-01-01', f'2023-01-01', freq='YS')

CPU times: user 12 μs, sys: 0 ns, total: 12 μs
Wall time: 21.2 μs


In [62]:
if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':
    ds_spear_a1_anom

In [63]:
%%time

if VARNAME == 'btm_arag_sat_ratio' or VARNAME == 'btm_o2':
    ds_nwa12_a1 = xr.open_mfdataset(f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_a1_ocean_{freq}.{VARNAME}-e*.nc' \
                                ,concat_dim='member',combine='nested')

else:
    ds_nwa12_a1 = xr.open_mfdataset(f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_a1_ocean_{freq}.{VARNAME}-e*.nc' \
                                ,concat_dim='member',combine='nested').drop_vars({'average_DT','time_bnds'})

if (VARNAME=='MLD_003' or VARNAME=='chlos') and freq=='monthly':

    if VARNAME=='MLD_003':
        selected_months=[1, 2, 3]
    else:
        selected_months=[3, 4, 5]  
        
    ds_nwa12_a1 = ds_nwa12_a1.sel(time=ds_nwa12_a1.time.dt.month.isin(selected_months)).groupby('time.year').mean().load().rename({'year':'time'})
    ds_nwa12_a1['time'] = pd.date_range(start=f'{ds_nwa12_a1.time.values[0]}', end=f'{ds_nwa12_a1.time.values[-1]}', freq='YS')
    
else:

    ds_nwa12_a1 = ds_nwa12_a1[f"{VARNAME}"].load()

ds_nwa12_a1_clim = ds_nwa12_a1.sel(time=slice(f'{clim_start}-01-01',f'{clim_end}-01-01')).mean(dim=['time'])
ds_nwa12_a1_anom = (ds_nwa12_a1.sel(time=slice(f'1965-01-01',f'2023-12-31')) - ds_nwa12_a1_clim)
ds_nwa12_a1_anom["time"] = pd.date_range(f'1965-01-01', f'2023-01-01', freq='YS')
ds_nwa12_a1_anom

CPU times: user 1.72 s, sys: 2.45 s, total: 4.17 s
Wall time: 5.94 s


<xarray.DataArray 'ssh' (member: 4, time: 59, yh: 845, xh: 775)> Size: 618MB
array([[[[        nan,         nan,         nan, ..., -0.03392185,
          -0.03411327, -0.03427488],
         [        nan,         nan,         nan, ..., -0.03364856,
          -0.0338198 , -0.03397867],
         [        nan,         nan,         nan, ..., -0.03345723,
          -0.03359978, -0.03373994],
         ...,
         [        nan,         nan,         nan, ...,  0.15812969,
           0.16101342,  0.16571164],
         [        nan,         nan,         nan, ...,  0.16198218,
           0.16480929,  0.1696583 ],
         [        nan,         nan,         nan, ...,  0.1658374 ,
           0.16830492,  0.1725893 ]],

        [[        nan,         nan,         nan, ..., -0.0321233 ,
          -0.03218556, -0.032245  ],
         [        nan,         nan,         nan, ..., -0.03243239,
          -0.03250493, -0.03259492],
         [        nan,         nan,         nan, ..., -0.03273821,
          -0.03280943, -0.03291219],
...
         [        nan,         nan,         nan, ...,  0.04695094,
           0.05042946,  0.05525088],
         [        nan,         nan,         nan, ...,  0.04734468,
           0.05024505,  0.05488229],
         [        nan,         nan,         nan, ...,  0.04479444,
           0.04650998,  0.05051351]],

        [[        nan,         nan,         nan, ...,  0.0363152 ,
           0.03579952,  0.03527954],
         [        nan,         nan,         nan, ...,  0.0360036 ,
           0.03546815,  0.0349506 ],
         [        nan,         nan,         nan, ...,  0.03572327,
           0.03517252,  0.03461039],
         ...,
         [        nan,         nan,         nan, ...,  0.04434824,
           0.04613841,  0.05129063],
         [        nan,         nan,         nan, ...,  0.04006755,
           0.04104054,  0.04593778],
         [        nan,         nan,         nan, ...,  0.03127885,
           0.03094602,  0.03517127]]]], dtype=float32)
Coordinates:
  * xh       (xh) float64 6kB -98.0 -97.92 -97.84 ... -36.24 -36.16 -36.08
  * yh       (yh) float64 7kB 5.273 5.352 5.432 5.511 ... 51.88 51.9 51.91 51.93
  * time     (time) datetime64[ns] 472B 1965-01-01 1966-01-01 ... 2023-01-01
Dimensions without coordinates: member

In [64]:
%%time

nwa12_model_grid = xarray.open_dataset('/work/vnk/nwa12/grid/ocean_static.nc').areacello
nwa12_model_grid = nwa12_model_grid.rename({'yh': 'lat', 'xh': 'lon'})

nwa12_hind_epu, mask_epu_nwa12_hind, mask_lmes_nwa12_hind = area_masked_weighted_average(
                        epu_shp_file, 
                        lmes_shp_file, 
                        nwa12_model_grid.copy(), 
                        ds_nwa12_hind_ly_anom.rename({'yh': 'lat', 'xh': 'lon'}).copy())
nwa12_hind_epu

CPU times: user 45.8 s, sys: 49.6 s, total: 1min 35s
Wall time: 1min 35s


<xarray.Dataset> Size: 73kB
Dimensions:  (epu: 6, lead: 10, time: 60, member: 5)
Coordinates:
  * member   (member) int64 40B 1 2 3 4 5
  * time     (time) datetime64[ns] 480B 1965-01-01 1966-01-01 ... 2024-01-01
Dimensions without coordinates: epu, lead
Data variables:
    ssh      (epu, lead, time, member) float32 72kB 0.02215 0.04542 ... nan

In [65]:
%%time

nwa12_hind_clim_epu, mask_epu_nwa12_hind_clim, mask_lmes_nwa12_hind_clim = area_masked_weighted_average(
                        epu_shp_file, 
                        lmes_shp_file, 
                        nwa12_model_grid.copy(), 
                        ds_nwa12_hind_ly_clim.rename({'yh': 'lat', 'xh': 'lon'}).copy())
nwa12_hind_clim_epu

CPU times: user 5.24 s, sys: 1.74 s, total: 6.98 s
Wall time: 7.02 s


<xarray.Dataset> Size: 1kB
Dimensions:  (epu: 6, lead: 10, member: 5)
Coordinates:
  * member   (member) int64 40B 1 2 3 4 5
Dimensions without coordinates: epu, lead
Data variables:
    ssh      (epu, lead, member) float32 1kB -0.6643 -0.6429 ... -0.6653 -0.6447

In [66]:
%%time

if VARNAME=='tos' or VARNAME=='tob':
    
    nwa12_hist_epu, mask_epu_nwa12_hist, mask_lmes_nwa12_hist = area_masked_weighted_average(
                            epu_shp_file, 
                            lmes_shp_file, 
                            nwa12_model_grid.copy(), 
                            ds_nwa12_hist_ym_anom.rename({'yh': 'lat', 'xh': 'lon'}).copy())
    nwa12_hist_epu

CPU times: user 11 μs, sys: 0 ns, total: 11 μs
Wall time: 19.3 μs


In [67]:
%%time

if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':

    spear_hind_grid = xarray.open_dataset('/work/vnk/spear_test_data/spear_a1/pp_ensemble/grids/ocean.static.nc').areacello
    spear_hind_grid = spear_hind_grid.rename({'yh': 'lat', 'xh': 'lon'})
    
    spear_hind_epu, mask_epu_spear_hind, mask_lmes_spear_hind = area_masked_weighted_average(
                            epu_shp_file, 
                            lmes_shp_file, 
                            spear_hind_grid, 
                            ds_spear_hind_ly_anom.rename({'yh': 'lat', 'xh': 'lon'}).copy())
    spear_hind_epu

CPU times: user 6 μs, sys: 6 μs, total: 12 μs
Wall time: 22.4 μs


In [68]:
%%time

if VARNAME=='tos' or VARNAME=='tob':

    spear_hist_grid = xarray.open_dataset('/work/vnk/spear_test_data/spear_hist/pp_ensemble/grids/ocean.static.nc').areacello
    spear_hist_grid = spear_hist_grid.rename({'yh': 'lat', 'xh': 'lon'})
    
    spear_hist_epu, mask_epu_spear_hist, mask_lmes_spear_hist = area_masked_weighted_average(
                            epu_shp_file, 
                            lmes_shp_file, 
                            spear_hist_grid, 
                            ds_spear_hist_ym_anom.rename({'yh': 'lat', 'xh': 'lon'}).copy())
    spear_hist_epu

CPU times: user 12 μs, sys: 0 ns, total: 12 μs
Wall time: 19.1 μs


In [69]:
%%time

nwa12_a1_epu, mask_epu_nwa12_a1, mask_lmes_nwa12_a1 = area_masked_weighted_average(
                        epu_shp_file, 
                        lmes_shp_file, 
                        nwa12_model_grid.copy(), 
                        ds_nwa12_a1_anom.rename({'yh': 'lat', 'xh': 'lon'}).copy())
nwa12_a1_epu

CPU times: user 19.3 s, sys: 6.7 s, total: 26 s
Wall time: 26.1 s


<xarray.DataArray 'ssh' (epu: 6, member: 4, time: 59)> Size: 6kB
array([[[ 0.0287273 ,  0.01346536,  0.00272655, ...,  0.0137191 ,
         -0.00489768,  0.04105325],
        [ 0.00685586,  0.00425359,  0.00524814, ...,  0.04217744,
          0.01981151,  0.06592421],
        [ 0.0338723 ,  0.01671052,  0.00824191, ...,  0.05351283,
          0.05723973,  0.07200869],
        [ 0.04076518,  0.03216823,  0.01136296, ...,  0.01522879,
          0.0318206 ,  0.0527359 ]],

       [[ 0.0354578 ,  0.02026681,  0.0007167 , ...,  0.00866871,
         -0.01464594,  0.02964999],
        [ 0.00791408,  0.01053366,  0.00696767, ...,  0.03421569,
          0.01898513,  0.05621027],
        [ 0.03402753,  0.02298662,  0.00438312, ...,  0.04884127,
          0.0580912 ,  0.06752668],
        [ 0.0450118 ,  0.0374136 ,  0.01123809, ...,  0.01003021,
          0.03162729,  0.0450994 ]],

       [[ 0.03386072,  0.02110904, -0.00148703, ...,  0.00621288,
         -0.01440091,  0.03118662],
...
        [ 0.04189311,  0.03418255,  0.00872376, ...,  0.01457698,
          0.03089135,  0.04578539]],

       [[ 0.03011063,  0.01485246,  0.00102519, ...,  0.01351119,
         -0.01010052,  0.03748306],
        [ 0.00538889,  0.00534787,  0.00447366, ...,  0.03991916,
          0.01858414,  0.06203461],
        [ 0.03217373,  0.01815809,  0.0044436 , ...,  0.05354824,
          0.05740412,  0.06983354],
        [ 0.04082073,  0.03281726,  0.00985735, ...,  0.01504551,
          0.03199194,  0.04991262]],

       [[ 0.01045392,  0.00514772,  0.00337233, ...,  0.01255829,
         -0.00129767,  0.02878352],
        [-0.015966  , -0.00613794, -0.00155854, ...,  0.0354743 ,
          0.01977131,  0.04112039],
        [ 0.01182744,  0.00696786,  0.00184471, ...,  0.0487705 ,
          0.05924934,  0.04755114],
        [ 0.02308811,  0.02324056,  0.00797067, ...,  0.02316627,
          0.04000044,  0.03579579]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 472B 1965-01-01 1966-01-01 ... 2023-01-01
Dimensions without coordinates: epu, member

In [70]:
%%time

nwa12_a1_clim_epu, mask_epu_nwa12_a1_clim, mask_lmes_nwa12_a1_clim = area_masked_weighted_average(
                        epu_shp_file, 
                        lmes_shp_file, 
                        nwa12_model_grid.copy(), 
                        ds_nwa12_a1_clim.rename({'yh': 'lat', 'xh': 'lon'}).copy())
nwa12_a1_clim_epu

CPU times: user 2.62 s, sys: 384 ms, total: 3.01 s
Wall time: 3.01 s


<xarray.DataArray 'ssh' (epu: 6, member: 4)> Size: 96B
array([[-0.6662141 , -0.65920866, -0.65072906, -0.6369089 ],
       [-0.7256192 , -0.7186218 , -0.7099676 , -0.6955094 ],
       [-0.7319568 , -0.7251632 , -0.7166018 , -0.7023568 ],
       [-0.6778327 , -0.67109627, -0.66262835, -0.648139  ],
       [-0.68103504, -0.67432827, -0.66585875, -0.6519064 ],
       [-0.67842704, -0.67372954, -0.6660142 , -0.65455395]],
      dtype=float32)
Dimensions without coordinates: epu, member

In [71]:
%%time

if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':

    spear_a1_epu, mask_epu_spear_a1, mask_lmes_spear_a1 = area_masked_weighted_average(
                            epu_shp_file, 
                            lmes_shp_file, 
                            spear_hind_grid, 
                            ds_spear_a1_anom.rename({'yh': 'lat', 'xh': 'lon'}).copy())
    spear_a1_epu

CPU times: user 2 μs, sys: 2 μs, total: 4 μs
Wall time: 8.34 μs


In [72]:
%%time

if VARNAME=='MLD_003':
    freq='JFM'
elif VARNAME=='chlos':
    freq='MAM'
else:
    freq='annual'
    
nwa12_hind_epu_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_hind_epu_{freq}_{VARNAME}.nc'
nwa12_hind_clim_epu_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_hind_clim_epu_{freq}_{VARNAME}.nc'
nwa12_hist_epu_file_path = f'/work/vnk/outdata_for_analysis/nwa12_hist_epu_{freq}_{VARNAME}.nc'
nwa12_a1_epu_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_a1_epu_{freq}_{VARNAME}.nc'
nwa12_a1_clim_epu_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/nwa12_a1_clim_epu_{freq}_{VARNAME}.nc'
spear_hind_epu_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_hind_epu_{freq}_{VARNAME}.nc'
spear_hist_epu_file_path = f'/work/vnk/outdata_for_analysis/spear_hist_epu_{freq}_{VARNAME}.nc'
spear_a1_epu_file_path = f'/work/vnk/outdata_for_analysis/post_202412/jra3q/spear_a1_epu_{freq}_{VARNAME}.nc'

if not os.path.exists(nwa12_hind_epu_file_path):
    nwa12_hind_epu.to_netcdf(path=nwa12_hind_epu_file_path, invalid_netcdf=False)
else:
    os.remove(nwa12_hind_epu_file_path)
    nwa12_hind_epu.to_netcdf(path=nwa12_hind_epu_file_path, invalid_netcdf=False)


if not os.path.exists(nwa12_hind_clim_epu_file_path):
    nwa12_hind_clim_epu.to_netcdf(path=nwa12_hind_clim_epu_file_path, invalid_netcdf=False)
else:
    os.remove(nwa12_hind_clim_epu_file_path)
    nwa12_hind_clim_epu.to_netcdf(path=nwa12_hind_clim_epu_file_path, invalid_netcdf=False)
    

if not os.path.exists(nwa12_a1_epu_file_path):
    nwa12_a1_epu.to_netcdf(path=nwa12_a1_epu_file_path, invalid_netcdf=False)
else:
    os.remove(nwa12_a1_epu_file_path)
    nwa12_a1_epu.to_netcdf(path=nwa12_a1_epu_file_path, invalid_netcdf=False)  


if not os.path.exists(nwa12_a1_clim_epu_file_path):
    nwa12_a1_clim_epu.to_netcdf(path=nwa12_a1_clim_epu_file_path, invalid_netcdf=False)
else:
    os.remove(nwa12_a1_clim_epu_file_path)
    nwa12_a1_clim_epu.to_netcdf(path=nwa12_a1_clim_epu_file_path, invalid_netcdf=False)  
    

if VARNAME=='tos' or VARNAME=='tob':

    if not os.path.exists(nwa12_hist_epu_file_path):
        nwa12_hist_epu.to_netcdf(path=nwa12_hist_epu_file_path, invalid_netcdf=False)
    else:
        os.remove(nwa12_hist_epu_file_path)
        nwa12_hist_epu.to_netcdf(path=nwa12_hist_epu_file_path, invalid_netcdf=False)

    if not os.path.exists(spear_hist_epu_file_path):
        spear_hist_epu.to_netcdf(path=spear_hist_epu_file_path, invalid_netcdf=False)
    else:
        os.remove(spear_hist_epu_file_path)
        spear_hist_epu.to_netcdf(path=spear_hist_epu_file_path, invalid_netcdf=False)
        
if VARNAME=='tos' or VARNAME=='tob' or VARNAME=='MLD_003':

    if not os.path.exists(spear_hind_epu_file_path):
        spear_hind_epu.to_netcdf(path=spear_hind_epu_file_path, invalid_netcdf=False)
    else:
        os.remove(spear_hind_epu_file_path)
        spear_hind_epu.to_netcdf(path=spear_hind_epu_file_path, invalid_netcdf=False)

    if not os.path.exists(spear_a1_epu_file_path):
        spear_a1_epu.to_netcdf(path=spear_a1_epu_file_path, invalid_netcdf=False)
    else:
        os.remove(spear_a1_epu_file_path)
        spear_a1_epu.to_netcdf(path=spear_a1_epu_file_path, invalid_netcdf=False)

CPU times: user 22.4 ms, sys: 5.3 ms, total: 27.7 ms
Wall time: 39.1 ms
